
## SageMaker managed training

- SageMaker에서 training을 진행하는 예시입니다.
- 기존의 training script를 일부만 수정하여 곧바로 SageMaker의 managed training 기능을 활용할 수 있습니다.
- `code` 디렉토리의 `train.py` 과 `local_train.py`를 비교 해 보세요.
- monai를 활용한 학습 코드 예시는 여기 [spleen_segmentation_3d notebook](https://github.com/Project-MONAI/tutorials/blob/master/3d_segmentation/spleen_segmentation_3d.ipynb) 를 참고해 주세요.

In [ ]:
%store -r

In [ ]:
s3_inputs

In [ ]:
import sagemaker 
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

In [ ]:
role = get_execution_role()
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = sess.default_bucket()

### Estimator 정의 및 학습

- SageMaker training container를 활용할 estimator를 정의합니다.
- 이 때 training script 및 framework 버전, instance type 및 개수 등을 정의해 주게 됩니다.
- 사용하는 instance 는 service quota 에서 가능한지 확인이 필요합니다. quota가 모자란 경우 resource limit 에 의한 에러가 발생합니다.

### 학습 코드

- 아래 예시 코드에서 학습은 로컬 디렉토리의 `code/train.py` 가 entry point 가 됩니다. 해당 파일을 확인 해 보세요.
- 필요한 패키지는 `code/requirements.txt` 에 명시함으로 학습이 시작됟기 전에 미리 설치됩니다. 이러한 방식으로 해결이 되지 않거나 설치 시간 등을 줄이고 싶은 경우 custom container를 만들어서 하면 됩니다.

In [ ]:

metrics=[
   {"Name": "train:average epoch loss", "Regex": "average loss: ([0-9\\.]*)"},
   {"Name": "train:current mean dice", "Regex": "current mean dice: ([0-9\\.]*)"},
   {"Name": "train:best mean dice", "Regex": "best mean dice: ([0-9\\.]*)"}
]

hyperparams = {
    "seed": 123,
    "lr": 0.001,
    # "epochs": 5,
    "epochs": 75
}

instance_type = "ml.g5.2xlarge"
# instance_type = "ml.p3.2xlarge"

estimator = PyTorch(source_dir="code",
                    entry_point="train.py",
                    role=role,
                    framework_version="1.13.1",
                    py_version="py39",
                    instance_count=1,
                    instance_type=instance_type,
                    hyperparameters=hyperparams,
                    metric_definitions=metrics)


# framework_version="1.6.0",
# py_version="py3",
### spot instance training ###
# use_spot_instances=True,
# max_run=2400,
# max_wait=2400


### Training 진행

- `fit()` 함수를 호출하여 학습을 시작할 수 있습니다.
- 여기서는 학습 데이터가 있는 s3경로를 주었습니다. 이 값은 dict 형태로 다양한 파라미터를 넘겨준 후 training job 내에서 사용될 수 있습니다.
- 자세한 내용은 [Estimator](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) 를 참고해 주세요.

In [ ]:
# estimator.fit(s3_inputs, wait=False)
estimator.fit(s3_inputs)